In [ ]:
!pip install gdown
import gdown
import zipfile
import os

file_id = '1CFgo7ucTGA4vd-NysR_kox7t9x0RW_Gc'
output = 'brats2017.zip'

gdown.download(id=file_id, output=output, quiet=False)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('brats2017')

os.remove(output)

Downloading...
From (original): https://drive.google.com/uc?id=1CFgo7ucTGA4vd-NysR_kox7t9x0RW_Gc
From (redirected): https://drive.google.com/uc?id=1CFgo7ucTGA4vd-NysR_kox7t9x0RW_Gc&confirm=t&uuid=f2481bdb-59a4-4843-9cf5-94a6ccfc43a8
To: /content/brats2017.zip
100%|██████████| 4.02G/4.02G [00:56<00:00, 71.8MB/s]


In [ ]:
import os
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

def find_representative_slice(seg_data):
    """
    Encuentra el slice más representativo basado en:
    1. Presencia de enhancing core (etiqueta 4)
    2. Centro de masa del tumor
    3. Mayor área de enhancing core
    """
    # Máscara binaria del enhancing core (etiqueta 4)
    enhancing_core = (seg_data == 4).astype(int)

    # Si no hay enhancing core, usar todo el tumor (etiquetas 1, 2, 4)
    if np.sum(enhancing_core) == 0:
        tumor_mask = np.isin(seg_data, [1, 2, 4])
        enhancing_core = tumor_mask

    # Calcular el centro de masa del tumor
    center_of_mass = ndimage.center_of_mass(enhancing_core)
    z_center = int(center_of_mass[2])

    # Buscar slices cercanos al centro con mayor área de enhancing core
    search_range = 5
    best_slice = z_center
    max_area = 0

    for z in range(z_center - search_range, z_center + search_range + 1):
        if 0 <= z < seg_data.shape[2]:
            slice_area = np.sum(enhancing_core[:, :, z])
            if slice_area > max_area:
                max_area = slice_area
                best_slice = z

    return best_slice

def normalize_slice(slice_data):
    """Normaliza el slice para visualización"""
    vmin = np.percentile(slice_data, 2)
    vmax = np.percentile(slice_data, 98)
    return (np.clip(slice_data, vmin, vmax) - vmin) / (vmax - vmin + 1e-8)

# =============================================================================
# Parámetro: MODALIDAD A PROCESAR (puedes cambiarlo por 'flair', 't1', 't1ce' o 't2')
# Si quieres procesar todas en bloque, reemplaza por: modalities = ['flair','t1','t1ce','t2']
modality = "t1ce"
# =============================================================================

# Rutas base
base_path = "brats2017/BRATS2017/Brats17TrainingData/HGG"
output_dir = f"Representative_Slices_{modality.upper()}"
os.makedirs(output_dir, exist_ok=True)

# Procesar cada paciente
for patient in os.listdir(base_path):
    patient_path = os.path.join(base_path, patient)
    if not os.path.isdir(patient_path):
        continue

    # Rutas de archivos de la modalidad seleccionada y de segmentación
    modality_file = os.path.join(patient_path, f"{patient}_{modality}.nii")
    seg_file = os.path.join(patient_path, f"{patient}_seg.nii")

    # Si falta alguno, saltar
    if not (os.path.exists(modality_file) and os.path.exists(seg_file)):
        print(f"[!] Skip {patient}: falta {modality}.nii o seg.nii")
        continue

    # Leer datos NIfTI
    modal_img = nib.load(modality_file)
    seg_img = nib.load(seg_file)
    modal_data = modal_img.get_fdata()
    seg_data = seg_img.get_fdata()

    # Encontrar slice representativo
    best_slice = find_representative_slice(seg_data)

    # Extraer slices
    modal_slice = modal_data[:, :, best_slice]
    seg_slice = seg_data[:, :, best_slice]

    # Crear visualización
    plt.figure(figsize=(12, 6))

    # Imagen de la modalidad seleccionada
    plt.subplot(1, 2, 1)
    plt.imshow(normalize_slice(modal_slice).T, cmap="gray", origin="lower")
    plt.title(f"{modality.upper()} - Slice {best_slice}")
    plt.axis("off")

    # Segmentación superpuesta sobre la misma modalidad
    plt.subplot(1, 2, 2)
    plt.imshow(normalize_slice(modal_slice).T, cmap="gray", origin="lower")

    # Superponer máscaras (solo enhancing core en rojo)
    colors = {4: (1, 0, 0)}  # Rojo: Enhancing Core
    for label, color in colors.items():
        mask = np.ma.masked_where(seg_slice.T != label, seg_slice.T)
        plt.imshow(mask, cmap="autumn", alpha=0.5, vmin=0, vmax=4, origin="lower")

    plt.title(f"Tumor Segmentation ({modality.upper()} - Slice {best_slice})")
    plt.axis("off")

    # Guardar resultado
    out_name = f"{patient}_{modality}_slice_{best_slice}.png"
    plt.savefig(os.path.join(output_dir, out_name),
                bbox_inches="tight", dpi=150)
    plt.close()

print("Proceso completado. Los slices se guardaron en:", output_dir)


[!] Skip .ipynb_checkpoints: falta t1ce.nii o seg.nii
Proceso completado. Los slices se guardaron en: Representative_Slices_T1CE


In [ ]:
!rm -rf Dicom_Slices_FLAIR/ Radiomics_Combined_AllHighGrade.csv Radiomics_CSVs/ Representative_Slices_FLAIR/ Representative_Slices_T1CE/ Radiomics_CSVs_3D/ Radiomics_CSVs_3DMask/ Radiomics_CSVs_Tumor3D/

In [ ]:
import os
import numpy as np
import SimpleITK as sitk
import pandas as pd
from scipy import ndimage
from radiomics import featureextractor

# ------------------------------------------------------------
def find_representative_slice(seg_array):
    """
    Encuentra el índice Z del slice más representativo:
    - Busca voxeles con label=4 (enhancing core). Si no hay, usa (1,2,4).
    - Calcula centro de masa y luego, en ±5 slices, el que tenga mayor área de label=4.
    """
    enhancing_core = 0
    if np.sum(enhancing_core) == 0:
        tumor_mask = np.isin(seg_array, [1, 2, 4]).astype(np.uint8)
        enhancing_core = tumor_mask

    com = ndimage.center_of_mass(enhancing_core)
    z_center = int(com[0])

    search_range = 5
    best_slice = z_center
    max_area = 0
    for z in range(z_center - search_range, z_center + search_range + 1):
        if 0 <= z < seg_array.shape[0]:
            area = np.sum(enhancing_core[z, :, :])
            if area > max_area:
                max_area = area
                best_slice = z

    return best_slice

# ------------------------------------------------------------
radiomics_settings = {
    'minimumROIDimensions': 2,
    'minimumROISize': None,
    'normalize': False,
    'normalizeScale': 1,
    'removeOutliers': None,
    'resampledPixelSpacing': None,
    'interpolator': 'sitkBSpline',
    'preCrop': False,
    'padDistance': 5,
    'distances': [1],
    'force2D': False,
    'force2Ddimension': 0,
    'resegmentRange': None,
    'label': 1,
    'additionalInfo': True,
    'binWidth': 25.0,
    'symmetricalGLCM': True,
    'correctMask': True
}

extractor = featureextractor.RadiomicsFeatureExtractor(**radiomics_settings)

# ------------------------------------------------------------
# Directorios base
base_path = "brats2017/BRATS2017/Brats17TrainingData"
radiomics_out_dir = "Radiomics_CSVs_3DMask"
os.makedirs(radiomics_out_dir, exist_ok=True)

# Procesar ambas clases de tumores: HGG (alto grado), LGG (bajo grado)
for tumor_type in ["HGG", "LGG"]:
    tumor_dir = os.path.join(base_path, tumor_type)
    if not os.path.exists(tumor_dir):
        continue

    is_high_grade = tumor_type == "HGG"

    for patient in os.listdir(tumor_dir):
        patient_dir = os.path.join(tumor_dir, patient)
        if not os.path.isdir(patient_dir):
            continue

        flair_path = os.path.join(patient_dir, f"{patient}_flair.nii")
        seg_path = os.path.join(patient_dir, f"{patient}_seg.nii")

        if not (os.path.exists(flair_path) and os.path.exists(seg_path)):
            print(f"[!] Se omite {patient}: faltan {patient}_flair.nii o {patient}_seg.nii")
            continue

        flair_sitk = sitk.ReadImage(flair_path)
        seg_sitk = sitk.ReadImage(seg_path)

        flair_array = sitk.GetArrayFromImage(flair_sitk)
        seg_array = sitk.GetArrayFromImage(seg_sitk)

        z_rep = find_representative_slice(seg_array)

        mask3d_array = np.zeros_like(seg_array, dtype=np.uint8)
        mask3d_array[z_rep, :, :] = (seg_array[z_rep, :, :] != 0).astype(np.uint8)

        mask3d_sitk = sitk.GetImageFromArray(mask3d_array)
        mask3d_sitk.CopyInformation(flair_sitk)

        print(f"Extrayendo radiomics 3D para {patient} ({tumor_type}), usando slice {z_rep}...")
        features_dict = extractor.execute(flair_sitk, mask3d_sitk)

        radiomics_data = {}
        for key, value in features_dict.items():
            if key.startswith("original_") or key.startswith("diagnostics_"):
                radiomics_data[key] = value

        df_features = pd.DataFrame.from_dict(radiomics_data, orient="index", columns=["Value"]).transpose()
        df_features.insert(0, "PatientID", patient)
        df_features.insert(1, "SliceIndex", z_rep)
        df_features.insert(2, "highGrade", is_high_grade)

        csv_filename = os.path.join(radiomics_out_dir, f"{patient}_radiomics_3Dmask.csv")
        df_features.to_csv(csv_filename, index=False)

        print(f"  → Radiomics CSV 3DMask guardado en: {csv_filename}\n")

print("=== PROCESO COMPLETADO: Radiomics para HGG y LGG con máscara 3D (slice significativo) ===")


INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1,

Extrayendo radiomics 3D para Brats17_2013_11_1 (HGG), usando slice 75...


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


  → Radiomics CSV 3DMask guardado en: Radiomics_CSVs_3DMask/Brats17_2013_11_1_radiomics_3Dmask.csv

Extrayendo radiomics 3D para Brats17_TCIA_309_1 (HGG), usando slice 91...


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


...

  → Radiomics CSV 3DMask guardado en: Radiomics_CSVs_3DMask/Brats17_TCIA_282_1_radiomics_3Dmask.csv



INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


Extrayendo radiomics 3D para Brats17_TCIA_307_1 (LGG), usando slice 49...


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm


  → Radiomics CSV 3DMask guardado en: Radiomics_CSVs_3DMask/Brats17_TCIA_307_1_radiomics_3Dmask.csv

=== PROCESO COMPLETADO: Radiomics para HGG y LGG con máscara 3D (slice significativo) ===


In [ ]:
import os
import glob
import pandas as pd

# 1) Definir carpeta donde están los CSV generados anteriormente
radiomics_dir = "Radiomics_CSVs_3DMask"

# 2) Buscar todos los archivos .csv dentro de esa carpeta
csv_files = glob.glob(os.path.join(radiomics_dir, "*.csv"))

# 3) Leer cada CSV (cada uno debería venir con sus columnas de PyRadiomics + PatientID, SliceIndex)
#    y añadir una columna "highGrade" = True (porque hasta ahora sólo hay pacientes high-grade).
dfs = []
for csv_path in csv_files:
    df = pd.read_csv(csv_path)
    dfs.append(df)

# 4) Concatenar todos los DataFrames en uno solo
combined_df = pd.concat(dfs, ignore_index=True)

cols = [c for c in combined_df.columns if c != "highGrade"] + ["highGrade"]
combined_df = combined_df[cols]

# 6) Guardar el resultado en un CSV unificado
output_path = "Radiomics_Combined_AllHighGrade.csv"
combined_df.to_csv(output_path, index=False)

print(f"→ CSV combinado guardado en: {output_path}")


→ CSV combinado guardado en: Radiomics_Combined_AllHighGrade.csv
